In [1]:
import torch
import torch.nn as nn
import argparse
import os
from torchsummary import summary
from torchvision import datasets, transforms
from kymatio.torch import Scattering2D
from tqdm import tqdm
import torch.nn.functional as F
from torch.optim.lr_scheduler import *
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [2]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                         std=[0.229, 0.224, 0.225])

train_transforms = [transforms.RandomHorizontalFlip(),
                    transforms.RandomCrop(32, 4),
                    transforms.ToTensor(),
                    normalize,
                   ]

In [3]:
train_set = datasets.CIFAR10(root=".data", train=True,
                                     transform=transforms.Compose(train_transforms),
                                     download=True)
test_set = datasets.CIFAR10(root=".data", train=False,
                                    transform=transforms.Compose(
                                        [transforms.ToTensor(), normalize]
                                    ))

Files already downloaded and verified


In [4]:
class ScatterLinear(nn.Module):
    def __init__(self, in_channels, hw_dims, input_norm=None, classes=10, **kwargs):
        super(ScatterLinear, self).__init__()
        self.Scattering2D = Scattering2D(J=2, shape=(32, 32))
        self.Scattering2D = self.Scattering2D.cuda()
        self.K = in_channels
        self.h = hw_dims[0]
        self.w = hw_dims[1]
        self.fc = None
        self.norm = None
        self.build(input_norm, classes=classes, **kwargs)

    def build(self, input_norm=None, num_groups=None, bn_stats=None, clip_norm=None, classes=10):
        self.fc = nn.Linear(self.K * self.h * self.w, classes)

        if input_norm is None:
            self.norm = nn.Identity()
        elif input_norm == "BatchNorm":
            self.norm = nn.BatchNorm2d(num_features=self.K,affine=False)

    def forward(self, x):
        x = self.Scattering2D(x)
        x = x.view(-1, 3 * 81*8*8)
        torch.flatten(x, start_dim=1)
        x = x.reshape(x.size(0), -1)
        x = self.fc(x)
        return x

In [5]:
model = ScatterLinear(in_channels=243,
                      hw_dims=[8,8],
                      input_norm='BatchNorm',
                      classes=10,
                      clip_norm=None)

In [6]:
train_loader = torch.utils.data.DataLoader(train_set, 
                                           batch_size=512,
                                           shuffle=True,
                                           num_workers=6, 
                                           drop_last=True)
test_loader = torch.utils.data.DataLoader(train_set, 
                                           batch_size=1024,
                                           shuffle=False,
                                           num_workers=6, 
                                           drop_last=False)

In [7]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.1,
                                    momentum=0.9,
                                    nesterov=False)
scheduler = MultiStepLR(optimizer, milestones=[30,80], gamma=0.1)

In [8]:
def train(model, train_loader, optimizer):
    model = model.to(device)
    model.train()
    num_examples = 0
    correct = 0
    train_loss = 0
    for batch_idx, (data, target) in tqdm(enumerate(train_loader)):
        data, target = data.to(device), target.to(device)
        output = model(data)
        #print(data.shape,output.shape, target.shape)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(target.view_as(pred)).sum().item()
        train_loss += F.cross_entropy(output, target, reduction='sum').item()
        num_examples += len(data)

    train_loss /= num_examples
    train_acc = 100. * correct / num_examples
    print(f'Train set: Average loss: {train_loss:.4f}, '
          f'Accuracy: {correct}/{num_examples} ({train_acc:.2f}%)')
    return train_loss, train_acc

In [9]:
def test(model, test_loader):
    device = next(model.parameters()).device
    
    model.eval()
    num_examples = 0
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in tqdm(test_loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            num_examples += len(data)

    test_loss /= num_examples
    test_acc = 100. * correct / num_examples

    print(f'Test set: Average loss: {test_loss:.4f}, '
          f'Accuracy: {correct}/{num_examples} ({test_acc:.2f}%)')

    return test_loss, test_acc

In [ ]:
Epochs = 100
train_loss_lst=[]
train_acc_lst=[]
val_loss_lst=[]
val_acc_lst=[]
for epoch in range(0, Epochs):
    print(f"\nEpoch: {epoch}")
    train_loss, train_acc = train(model, train_loader, optimizer)
    test_loss, test_acc = test(model, test_loader)
    train_loss_lst.append(train_loss)
    train_acc_lst.append(train_acc)
    val_loss_lst.append(test_loss)
    val_acc_lst.append(test_acc)
    print(train_loss_lst)
    print(train_acc_lst)
    print(val_loss_lst)
    print(val_acc_lst)
    scheduler.step()


Epoch: 0


97it [00:13,  7.20it/s]


Train set: Average loss: 1.7494, Accuracy: 18760/49664 (37.77%)


100%|██████████| 49/49 [00:13<00:00,  3.54it/s]

Test set: Average loss: 1.5840, Accuracy: 21958/50000 (43.92%)
[1.7493649168112844]
[37.77384020618557]
[1.5840436083984375]
[43.916]

Epoch: 1


...

100%|██████████| 49/49 [00:13<00:00,  3.59it/s]

Test set: Average loss: 1.1705, Accuracy: 30107/50000 (60.21%)
[1.7493649168112844, 1.5285044429228478, 1.4675812794990146, 1.4329862373391378, 1.3974224513339013, 1.3838434108753794, 1.3590353874816108, 1.3544005942098873, 1.3355169259395796, 1.3249074429580845, 1.316713673552287, 1.3182192672159254, 1.3052511104603404, 1.3050669505424106, 1.3003858106652486, 1.2850659090219085, 1.291957246888544, 1.2956515771826518, 1.271739296077453, 1.2726994752883911, 1.2761381508148824, 1.269474671059048, 1.2737478066965477, 1.2635394125869595, 1.2585786015717024, 1.2615706994361484, 1.2608905330146711, 1.2485677003860474, 1.2452633528365302, 1.238563591671973, 1.182949243132601, 1.1711935751216929, 1.1705198300253485, 1.1709989702578671, 1.1701623557769145]
[37.77384020618557, 46.04341172680412, 48.49387886597938, 49.66575386597938, 51.230267396907216, 51.78398840206186, 52.88740335051546, 52.86122744845361, 53.54381443298969, 53.87403350515464, 54.274726159793815, 54.23244201030928, 54.49017396


97it [00:13,  7.17it/s]


Train set: Average loss: 1.1694, Accuracy: 30013/49664 (60.43%)


 84%|████████▎ | 41/49 [00:11<00:02,  3.76it/s]